# Machine learning: correlated multivariate profiles

In [ ]:
# Goal: to improve the prediction of a simple ml for predicting radiation flux.
# Background: the number of samples (named profiles/columns in the data) are scarce, can we generate
#             a larger set of random profiles which stll capture the correclation between differnt quantities?

In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import numpy as np
from scipy.constants import Stefan_Boltzmann
import xarray as xr
import gdown
from tarfile import TarFile

import matplotlib.pyplot as plt

import synthia as syn

In [ ]:
def compute_cloud_optical_depth(ds: xr.Dataset) -> xr.DataArray:
    # Constants
    g = 9.81 # m/s²
    rho_liquid = 1000 # kg/m³
    rho_ice = 917 # kg/m³

    delta_pressure = ds['pressure_hl'].diff('half_level').rename('delta_pressure')
    delta_pressure = delta_pressure.rename({'half_level': 'level'})
    
    cloud_optical_depth = (ds['q_liquid'] / (rho_liquid * ds['re_liquid']) +\
                     ds['q_ice'] / (rho_ice * ds['re_ice']) ) * delta_pressure / g
    return cloud_optical_depth

In [ ]:
def compute_layer_optical_depth(delta_pressure_fl: xr.DataArray, ext_coeff_fl: xr.DataArray):
    """ Compute the layer optical depth from a profile of extinsion coefficients
    """
    layer_optical_depth = ext_coeff_fl * delta_pressure_fl
    return layer_optical_depth

In [ ]:
def compute_emissivity(delta_pressure_fl: xr.DataArray, ext_coeff_fl: xr.DataArray) -> xr.DataArray:
    diffusivity_factor = 1/np.cos(np.radians(53)) 
    layer_optical_depth = compute_layer_optical_depth(delta_pressure_fl, ext_coeff_fl)
    emissivity = 1 - np.exp(-diffusivity_factor * layer_optical_depth)
    return emissivity

In [ ]:
def compute_plank_func(temperature: xr.DataArray) -> xr.DataArray:
    plank_func = Stefan_Boltzmann * temperature**4
    return plank_func

In [ ]:
def compute_lw_up_boa(skin_temperature: xr.DataArray,
                      lw_emissivity: xr.DataArray) -> xr.DataArray:
    """Compute the upwelling longawe flux at BOA
    """
    lw_up_boa = lw_emissivity * compute_plank_func(skin_temperature)
    lw_up_boa = lw_up_boa.rename('flux_lw_up_boa')
    lw_up_boa.attrs = {'long_name': 'Upward logwave radiation at BOA', 'units': 'W/m2'}
    return lw_up_boa

In [ ]:
def compute_lw_up_profile(temperature_fl: xr.DataArray,
                          delta_pressure_fl: xr.DataArray,
                          ext_coeff_fl: xr.DataArray,
                          flux_at_boa: xr.DataArray) -> xr.DataArray:
    """Compute the upwelling longawe flux profile to TOA given flux at BOA
    """
    # Array to store computed fluxes
    n_column = len(temperature_fl.column)
    n_level = len(temperature_fl.level)
    da_flux = xr.DataArray(
        np.zeros((n_column, n_level+1)),
        dims=('column', 'half_level'), # n_half_level = n_level + 1
        name='flux_up_hl',
        attrs = {'long_name': 'Upward logwave radiation', 
                 'units': 'W/m2'}
    )

    # Assign BC at BOA
    da_flux[:, -1] = flux_at_boa
    
    # Precompute emissivity and plank function as these are independent
    emissivity = compute_emissivity(delta_pressure_fl, ext_coeff_fl)
    plank_function = compute_plank_func(temperature_fl)

    n_half_level = list(range(len(da_flux.half_level))) 
    # Interate over half levels to TOA
    # Revert as TOA is at index zero.
    for i in range(da_flux.shape[1] - 1, 0, -1):
        da_flux[:, i-1] = da_flux[:, i] * (1 - emissivity[:, i-1]) + plank_function[:, i-1] * emissivity[:, i-1]
    return da_flux

In [ ]:
def compute_lw_up(temperature_fl: xr.DataArray,
                  delta_pressure_fl: xr.DataArray,
                  ext_coeff_fl: xr.DataArray,
                  skin_temperature: xr.DataArray,
                  lw_emissivity: xr.DataArray) -> xr.DataArray:
    """Wrapper function to cumpute the full profile from BOA to TOA
    """
    flux_at_boa = compute_lw_up_boa(skin_temperature, lw_emissivity)
    lw_up = compute_lw_up_profile(ds, flux_at_boa, opt_depth)
    return lw_up

## Compute upward longwave radiation from temperature and optical depth

Here we use the functions we defined earlier to compute and plot the upward longwave radiation from temperature profiles and optical depth. x-axis indicates pressure levels where 0 is TOA and 137 is BOA. 

In [ ]:
def compute_ext_coeff(pressure_fl, opt_depth):
    """ Compute the extinsion coefficient for
    the atmosphere given atmospheric pressure
    and atmospheric optical depth
    """
    ATM_SCALE_HEIGHT = 300000
    A = opt_depth / ATM_SCALE_HEIGHT
    ext_coeff = A * np.exp(-ATM_SCALE_HEIGHT / pressure_fl)
    return ext_coeff

In [ ]:
THIS_DIR = Path.cwd()
ds_input = xr.open_dataset(THIS_DIR.parents[1] / 'data' / 'nwp_saf_profiles_in.nc')

ds_input['delta_pressure_fl'] = ds_input['pressure_hl'].diff('half_level').rename(half_level='level')

opt_depth = 30
ds_input['ext_coeff_fl'] = compute_ext_coeff(ds_input['pressure_fl'], opt_depth)

input_relevant = ['temperature_fl', # for plank fuction 
                  'delta_pressure_fl', # for cloud optical depth
                  'ext_coeff_fl', # for cloud optical depth
                  'skin_temperature', # for flux at BOA
                  'lw_emissivity', # for flux at BOA
                  #'cloud_fraction' # FIXME: not currently used but of interest as between 0 and 1.
]

ds_input = ds_input[input_relevant]
ds_input

In [ ]:
flux_at_boa = compute_lw_up_boa(ds_input['skin_temperature'],
                                ds_input['lw_emissivity'])

ds_output = compute_lw_up_profile(ds_input['temperature_fl'],
                                  ds_input['delta_pressure_fl'],
                                  ds_input['ext_coeff_fl'],
                                  flux_at_boa)

ds_output.mean('column').plot()
ds_input_output = xr.merge([ds_input, ds_output])

In [ ]:
def plot_profile(ds, n_profiles):
    for idx in np.random.choice(ds.column, n_profiles):
        fig, axs = plt.subplots(1,2, figsize=(5*2,4))
        ds['temperature_fl'].isel(column=idx).plot(ax=axs[0], c='r')
        ds['flux_up_hl'].isel(column=idx).plot(ax=axs[1], c='k')
        plt.show()

In [ ]:
plot_profile(ds_input_output, 10)

## Machine learning

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso

In [ ]:
def compute_norm_stats(ds):
    stats = {
        name : {
            'mean' : ds[name].mean(),
            'std' : ds[name].std()
        } for name in ds
    }
    return stats

def normalize_inputs(ds, norm_stats):

    def compute_z_score(ds, stats):
        return (ds - stats['mean']) / stats['std']

    ds_norm = xr.zeros_like(ds)

    # These are already in reasonable scale O(1).
    quantity_no_norm = ['lw_emissivity']
    for quantity in list(ds_norm):
        if quantity in quantity_no_norm:
            ds_norm[quantity] = ds[quantity]
            print(f'Skipping normalization for: {quantity}')
        else:
            ds_norm[quantity] = compute_z_score(ds[quantity], norm_stats[quantity]) 
    return ds_norm

In [ ]:
def plot_normalized_inputs(ds):
    fig, ax = plt.subplots(1,2, figsize=(15,5))
    for quantity in list(ds):
        if len(ds[quantity].shape) == 1: # scalars
            ds[quantity].plot.hist(ax=ax[0], label=quantity, alpha=0.3)
            ax[0].set_ylabel('Count')
            ax[0].set_xlabel('Range')
            ax[0].legend()
        elif len(ds[quantity].shape) == 2: # profiles
            ds[quantity].mean('column').plot(ax=ax[1], label=quantity)
            ax[1].set_ylabel('Normilized range (Z score)')
            ax[1].set_xlabel('Vertical level')
            ax[1].set_title('Mean profiles')
            ax[1].legend()
            ax
        else:
            raise RuntimeError('Number of dims not supported')

In [ ]:
X_true = xr.merge([ds_input, flux_at_boa])
X_true

In [ ]:
norm_stats = compute_norm_stats(X_true)
X_true_norm = normalize_inputs(X_true, norm_stats)
X_true_norm = X_true_norm.drop(['delta_pressure_fl', 'lw_emissivity', 'skin_temperature'])
plot_normalized_inputs(X_true_norm)

In [ ]:
X_true_norm = X_true_norm
X_true_norm

In [ ]:
# Flatten
X_true_stacked = X_true_norm.to_stacked_array("feature", sample_dims=['column'])
y_true = ds_output.sel(half_level=slice(0,-1))
display(X_true_stacked, y_true)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_true_stacked, y_true, test_size=0.33, random_state=42)

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split

reg = Ridge().fit(X_train, y_train)

In [ ]:
idx = 84
plt.plot(range(137), y_test[idx, :])
plt.plot(range(137), reg.predict(X_test[idx:idx+1])[0, :])

In [ ]:
plt.scatter(y_test[:, :], reg.predict(X_test)[:, :])

# Copula

In [ ]:
from synthia.util import to_stacked_array, to_unstacked_dataset
import pyvinecopulib as pv

In [ ]:
# data -> split test_train -> copula -> nomrmalize -> 
# 2. Normilize 
X_true

In [ ]:
# Load data
X_true = X_true[['temperature_fl', 'ext_coeff_fl', 'flux_lw_up_boa']]
y_true = y_true

# Flatten
X_true_stacked = X_true.to_stacked_array('feature', sample_dims=['column'])
y_true_stacked = y_true.stack(feature=('half_level',))

# Split train/test
X_true_stacked_train, X_true_stacked_test, \
y_true_stacked_train, y_true_stacked_test = train_test_split(X_true_stacked, 
                                                             y_true_stacked,
                                                             test_size=0.33,
                                                             random_state=42)

display(X_true_stacked_train, y_true_stacked_train)

In [ ]:
# Combine x and y for copula

X_true_train = X_true_stacked_train.to_unstacked_dataset('feature')
y_true_train = y_true_stacked_train.unstack('feature')

X_y_true_train = xr.merge([X_true_train, y_true_train])
X_y_true_train_stacked, stack_info = to_stacked_array(X_y_true_train)

X_y_true_train_stacked

In [ ]:
parameterizer = syn.QuantileParameterizer(n_quantiles=100)

generator = syn.CopulaDataGenerator(verbose=True)

#ctrl = pv.FitControlsVinecop(family_set=[pv.BicopFamily.tll], select_trunc_lvl=True)
#generator.fit(X_y_true_train_stacked, copula=syn.VineCopula(controls=ctrl), parameterize_by=parameterizer)
generator.fit(X_y_true_train_stacked, copula=syn.GaussianCopula(), parameterize_by=None)

In [ ]:
N_SAMPLES = X_y_true_train_stacked.shape[0]
X_y_synth_train_stacked = generator.generate(n_samples=N_SAMPLES, uniformization_ratio=0, stretch_factor=1)

In [ ]:
X_y_synth_train = to_unstacked_dataset(X_y_synth_train_stacked, stack_info)
X_y_synth_train

In [ ]:
X_synth_train = X_y_synth_train
X_synth_train = X_y_synth_train.drop('flux_up_hl')
y_synth_train = X_y_synth_train[['flux_up_hl']]
display(X_synth_train, y_synth_train)

In [ ]:
for column in np.random.choice(X_y_synth_train.column, 100): 
    X_y_synth_train['flux_up_hl'].sel(column=column).plot()

In [ ]:
for column in np.random.choice(y_true.column, 100): 
    X_true['ext_coeff_fl'].sel(column=column).plot()

In [ ]:
for column in np.random.choice(y_true.column, 100): 
    y_true.sel(column=column).plot()

In [ ]:
#X_synth_train_stacked, stack_info = to_stacked_array(X_synth_train)
#y_synth_train_stacked, stack_info = to_stacked_array(y_synth_train)
X_synth_train_norm = normalize_inputs(X_synth_train, norm_stats)
X_synth_train_norm_stacked = X_synth_train_norm.to_stacked_array('feature', sample_dims=['column'])
y_synth_train_stacked = y_synth_train.to_stacked_array('feature', sample_dims=['column'])
reg = Ridge().fit(X_synth_train_norm_stacked, y_synth_train_stacked)

In [ ]:
idx = 5
plt.plot(range(137), y_test[idx, :], label='true')
plt.plot(range(137), reg.predict(X_test[idx:idx+1])[0, :])
plt.legend()

In [ ]:
reg = Ridge().fit(X_train, y_train)